# Implementing IOU, NMS, and mAP

In this notebook, I have implemented the following utilities for Object Detection:

- **Intersection over Union (IOU) Calculation**: A metric used to evaluate the accuracy of an object detector by measuring the overlap between the predicted bounding box and the ground truth bounding box.
- **Non-Maximum Suppression (NMS)**: An algorithm used to select the best bounding box when multiple bounding boxes are predicted for a single object by suppressing all other boxes that have a high overlap (IOU) with the selected box.
- **Mean Average Precision (mAP) Computation**: A metric used to evaluate the performance of an object detector by calculating the average precision for each class and then averaging over all classes. Implemented using both the Area Method and 11 points interpolation.

These implementations may not be the fastest or most efficient, but they provide a clear understanding of the underlying concepts and algorithms.

In [ ]:
import numpy as np

**IOU - Intersection over Union**

IOU is a metric used to evaluate the accuracy of an object detector on a particular dataset. It measures the overlap between two bounding boxes: the predicted bounding box and the ground truth bounding box. The formula for IOU is:

$$ \text{IOU} = \frac{\text{Area of Overlap}}{\text{Area of Union}} $$

In [ ]:
def get_iou(det, gt):
	# det = [x1, y1, x2, y2] where x1, y1 is the top left corner and x2, y2 is the bottom right corner
	# gt = [x1, y1, x2, y2] where x1, y1 is the top left corner and x2, y2 is the bottom right corner
	det_x1, det_y1, det_x2, det_y2 = det
	gt_x1, gt_y1, gt_x2, gt_y2 = gt

	overlap_x1 = max(det_x1, gt_x1)
	overlap_y1 = max(det_y1, gt_y1)
	overlap_x2 = min(det_x2, gt_x2)
	overlap_y2 = min(det_y2, gt_y2)

	if overlap_x1 >= overlap_x2 or overlap_y1 >= overlap_y2:
		return 0
	
	overlap_area = (overlap_x2 - overlap_x1) * (overlap_y2 - overlap_y1)
	det_area = (det_x2 - det_x1) * (det_y2 - det_y1)
	gt_area = (gt_x2 - gt_x1) * (gt_y2 - gt_y1)
	iou = overlap_area / (det_area + gt_area - overlap_area + 1e-6)
	return iou

**NMS - Non-Maximum Supression**

NMS is an algorithm used to select the best bounding box when multiple bounding boxes are predicted for a single object. It works by selecting the bounding box with the highest confidence score and suppressing all other boxes that have a high overlap (IOU) with the selected box. The steps for NMS are:

1. Select the bounding box with the highest confidence score.
2. Remove all other bounding boxes that have an IOU greater than a threshold with the selected box.
3. Repeat steps 1 and 2 for the remaining boxes.

In [ ]:
def nms(dets, threshold=0.5):
	# dets: [[x1, y1, x2, y2, score], ...]
	sorted_dets = sorted(dets, key=lambda x: x[4], reverse=True)
	keep_dets = []
	while len(sorted_dets)>0:
		keep_dets.append(sorted_dets[0])
		iou = [get_iou(sorted_dets[0][:4], det[:4]) for det in sorted_dets[1:]]
		sorted_dets = [sorted_dets[i] for i in range(1, len(sorted_dets)) if iou[i-1] <= threshold]
	return keep_dets


**mAP - mean Average Precision**

mAP is a metric used to evaluate the performance of an object detector. It calculates the average precision for each class and then averages over all classes. The steps to calculate mAP are:

1. Calculate precision and recall for each class at different IOU thresholds.
2. Compute the average precision (AP) for each class by integrating the precision-recall curve.
3. Calculate the mean of the average precision values for all classes to get the mAP.

In [ ]:
def compute_ap(det_boxes, gt_boxes, iou_threshold=0.5, method='area'):
	'''
	det_boxes = [
		# image 1
		{
			'person': [[x1, y1, x2, y2, score], ...],
			'car': [[x1, y1, x2, y2, score], ...],
			'image_id': 'image1'
		},
		# image 2
		{
			'person': [[x1, y1, x2, y2, score], ...],
			'car': [[x1, y1, x2, y2, score], ...],
			'image_id': 'image2'
		},
		...
	]

	gt_boxes = [
		# image 1
		{
			'person': [[x1, y1, x2, y2], ...],
			'car': [[x1, y1, x2, y2], ...],
			'image_id': 'image1'
		},
		# image 2
		{
			'person': [[x1, y1, x2, y2], ...],
			'car': [[x1, y1, x2, y2], ...],
			'image_id': 'image2'
		},
		...
	]

	iou_threshold: iou threshold for nms
	method: 'area' or 'point'
	'''

	# get all class labels
	gt_labels = set()
	for img_gt_box in gt_boxes:
		for cls_label in img_gt_box:
			gt_labels.add(cls_label)

	aps = []
	for idx, cls_label in enumerate(gt_labels):
		print(f"Computing AP for {cls_label}")

		cls_dets = []
		# class_dets = [[image_id, [x1, y1, x2, y2, score]], ...]
		for img_det_boxes in det_boxes:
			for cls_img_det_box in img_det_boxes[cls_label]:
				cls_dets.append([img_det_boxes["image_id"], cls_img_det_box])
		cls_dets = sorted(cls_dets, key=lambda x: x[1][4], reverse=True)

		cls_gt_boxes = {}
		for img_gt_box in gt_boxes:
			cls_gt_boxes[img_gt_box["image_id"]] = img_gt_box[cls_label]

		gt_matched = {img_gt_box["image_id"]: [False]*len(img_gt_box[cls_label]) for img_gt_box in gt_boxes}
		num_gt = sum([len(img_gt_box[cls_label]) for img_gt_box in gt_boxes])

		tp = [0]*len(cls_dets)
		fp = [0]*len(cls_dets)

		# for each prediction
		for idx, cls_det in enumerate(cls_dets):
			img_id, img_det = cls_det
			# get all ground truth boxes for this image and class
			if img_id not in cls_gt_boxes:
				fp[idx] = 1
				continue
			img_cls_gt_boxes = cls_gt_boxes[img_id]

			# get iou with all ground truth boxes
			iou = [get_iou(img_det[:4], img_gt) for img_gt in img_cls_gt_boxes]
			max_iou = max(iou)
			max_iou_idx = iou.index(max_iou)

			if max_iou >= iou_threshold and not gt_matched[img_id][max_iou_idx]:
				tp[idx] = 1
				gt_matched[img_id][max_iou_idx] = True
			else:
				fp[idx] = 1

		tp_cumsum = np.cumsum(tp)
		fp_cumsum = np.cumsum(fp)
		eps = 1e-6
		recalls = tp_cumsum / np.maximum(num_gt, eps)
		precisions = tp_cumsum / np.maximum(tp_cumsum + fp_cumsum, eps)

		if method=='area':
			recalls = np.concatenate([[0.0], recalls, [1.0]])
			precisions = np.concatenate([[0.0], precisions, [0.0]])

			for i in range(len(precisions)-2, -1, -1):
				precisions[i] = max(precisions[i], precisions[i+1])

			ap = 0
			for i in range(len(recalls)-1):
				ap += (recalls[i+1] - recalls[i]) * precisions[i+1]

		elif method=='interpolation':
			ap = 0
			for t in np.arange(0, 1.1, 0.1):
				prec = max([precisions[i] for i in range(len(precisions)) if recalls[i] >= t] + [0])
				ap += prec / 11

		else:
			raise ValueError("method must be 'area' or 'interpolation'")
		
		print(f"AP for {cls_label} with threshold {iou_threshold}= {ap}")
		aps.append(ap)

	return aps


In [ ]:
# compute map by mean with different iou thresholds
def compute_map(det_boxes, gt_boxes, iou_thresholds=np.arange(0.5, 1.0, 0.05), method='area'):
	aps = []
	for iou_threshold in iou_thresholds:
		cls_aps = compute_ap(det_boxes, gt_boxes, iou_threshold=iou_threshold, method=method)
		aps.append(cls_aps)
	aps = np.array(aps)
	map = np.mean(aps, axis=0)
	return map
